In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path

In [ ]:
def evaluate_and_save(y_true, y_pred, model_name, impl_name, out_filename):
    """
    Evaluate forecast accuracy (MAE, RMSE, MAPE, OPE, R²) and save results
    into a CSV file inside the project's `data/models` folder.

    If replace_existing=True:
        - Any existing row with the same (Model, Impl) is replaced.
    If replace_existing=False:
        - New rows are appended (old ones kept).

    Parameters
    ----------
    y_true : array-like
        Ground truth values.

    y_pred : array-like
        Model predictions.

    model_name : str
        Model identifier, e.g. "Naive", "RandomForestRegressor", "LSTM".

    impl_name : str
        Model family, e.g. "baseline", "statsmodels", "ml", "neural".

    out_filename : str
        File name of output CSV (saved in ../data/models/).

    Returns
    -------
    dict
        Dictionary of computed metrics.
    """

    # Convert inputs
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)

    eps = 1e-12

    # Metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None)))
    ope = np.abs(y_pred.sum() - y_true.sum()) / np.clip(np.abs(y_true.sum()), eps, None)
    r2 = r2_score(y_true, y_pred)

    result = {
        "Model": model_name,
        "Impl": impl_name,
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "OPE": ope,
        "R2": r2,
    }

    out_path = Path("../data/models")
    out_path.mkdir(parents=True, exist_ok=True)
    csv_path = out_path / out_filename

    # If file exists: load + delete old row for the same model
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        df = df[~((df["Model"] == model_name) & (df["Impl"] == impl_name))] # Remove old entry if model and impl match
        df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)
    else:
        df = pd.DataFrame([result])

    # Sort and save
    df = df.sort_values("MAE", ascending=True)
    df.to_csv(csv_path, index=False)

    return result

In [ ]:
def load_best_models(csv_path):
    """
    Load a CSV file containing forecast evaluation results and return
    the best model per family based on MAE.

    The CSV must contain at least the following columns:
    - Model   (name of the model)
    - MAE     (mean absolute error)
    - Impl    (model family: baseline, statsmodels, ml, neural, tirex, ...)
    
    Parameters
    ----------
    csv_path : str or Path
        Path to the CSV file containing model evaluation results.
    
    Returns
    -------
    pd.DataFrame
        A dataframe containing one row per model family, representing the
        best-performing model (lowest MAE) in each family.
        
        Columns:
            - Impl     (family)
            - Model    (best model name)
            - MAE, RMSE, MAPE, OPE, R2  (evaluation metrics)
    
    Example
    -------
    best = load_best_models("data/models/baseline_results.csv")

    or also to combine all of them

    all_best = pd.concat([
    load_best_models("data/models/baseline_results.csv"),
    load_best_models("data/models/statsmodels_results.csv"),
    load_best_models("data/models/ml_models_results.csv"),
    load_best_models("data/models/neural_models_results.csv"),
    ], ignore_index=True)
    """

    csv_path = Path(csv_path)

    if not csv_path.exists():
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    df = pd.read_csv(csv_path)

    if "Impl" not in df.columns:
        raise ValueError("CSV must contain a column 'Impl' for model family.")

    # Get best model per family (lowest MAE)
    best_per_family = (
        df.sort_values("MAE", ascending=True)
          .groupby("Impl", as_index=False)
          .head(1)
          .reset_index(drop=True)
    )

    return best_per_family